In [16]:
%pip install -r requirements.txt
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -------------------------------- ------- 8.9/11.0 MB 60.5 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 54.5 MB/s eta 0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
print("main ipynb")

main ipynb


In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import importlib
import Utils.wsi as wsi

# importlib.reload(Utils.wsi)

print("Output: ", wsi.load_wsi("data", "data/metadata.csv"))

Loading WSIs:   0%|          | 0/1 [00:00<?, ?it/s]

Extracted 10 patches...
Extracted 20 patches...
Extracted 30 patches...
Extracted 40 patches...
Extracted 50 patches...
Extracted 60 patches...
Extracted 70 patches...
Extracted 80 patches...
Extracted 90 patches...
Extracted 100 patches...


Loading WSIs: 100%|██████████| 1/1 [02:37<00:00, 157.32s/it]

Output:  [([array([[[219, 204, 227, 255],
        [210, 201, 224, 255],
        [211, 197, 213, 255],
        ...,
        [224, 210, 225, 255],
        [211, 200, 216, 255],
        [223, 213, 229, 255]],

       [[220, 205, 225, 255],
        [211, 203, 223, 255],
        [217, 203, 222, 255],
        ...,
        [217, 205, 219, 255],
        [214, 204, 219, 255],
        [236, 226, 241, 255]],

       [[227, 212, 228, 255],
        [219, 211, 226, 255],
        [218, 204, 228, 255],
        ...,
        [208, 195, 210, 255],
        [214, 200, 215, 255],
        [230, 217, 232, 255]],

       ...,

       [[223, 209, 226, 255],
        [218, 203, 222, 255],
        [228, 210, 229, 255],
        ...,
        [231, 194, 214, 255],
        [234, 197, 209, 255],
        [225, 187, 207, 255]],

       [[229, 211, 226, 255],
        [221, 205, 222, 255],
        [219, 206, 225, 255],
        ...,
        [230, 193, 211, 255],
        [229, 186, 203, 255],
        [220, 185, 202, 255]],



In [ ]:
%pip install openslide-bin

Using cached openslide_bin-4.0.0.8-py3-none-win_amd64.whl (4.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Path to your CSV file
csv_path = "data/Metadata.csv"

# Load data
df = pd.read_csv(csv_path)

# Drop rows with missing values (optional, or use imputation)
df = df.dropna()

df['Oncotype DX Breast Recurrence Score'] = (df['Oncotype DX Breast Recurrence Score'] > 25).astype(int)

# Example: target column = 'Label' (0 = control, 1 = disease)
target_col = 'Oncotype DX Breast Recurrence Score'

df = df.drop(columns=['svs_name', 'id'], errors='ignore')

# Split features and labels
X = df.drop(columns=[target_col])
y = df[target_col]

# One-hot encode categorical features automatically
X = pd.get_dummies(X, drop_first=True)

# Standardize numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y.values, test_size=0.2, random_state=42
)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

class ClinicalDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = ClinicalDataset(X_train, y_train)
test_dataset = ClinicalDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegressionModel(X_train.shape[1])

criterion = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_label = (y_pred >= 0.5).float()

accuracy = (y_pred_label == y_test).float().mean()
print(f"Test Accuracy: {accuracy.item():.4f}")


Epoch 10/100, Loss: 0.6039
Epoch 20/100, Loss: 0.4349
Epoch 30/100, Loss: 0.4036
Epoch 40/100, Loss: 0.3493
Epoch 50/100, Loss: 0.4064
Epoch 60/100, Loss: 0.3431
Epoch 70/100, Loss: 0.5198
Epoch 80/100, Loss: 0.2575
Epoch 90/100, Loss: 0.4632
Epoch 100/100, Loss: 0.3788
Test Accuracy: 0.8223
